HW6 Мельчук А.Б.

## Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
max_words = 2000
max_len = 40
num_classes = 1

# Training
epochs = 10
batch_size = 512
print_batch_n = 100

In [4]:
data = pd.read_excel("./отзывы за лето.xls")

In [5]:
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


### Предобработка

In [6]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [7]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in exclude]
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)
data = data[data['Rating'] != 3]
data['target'] = data['Rating'] > 3

In [8]:
data['target'] = data['target'].astype(int)
data.head()

,Rating,Content,Date,text,target
0,5,It just works!,2017-08-14,it just works,1
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,в целое удобноной приложениеиз минус хотеть сл...,1
2,5,Отлично все,2017-08-14,отлично весь,1
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,стать зависать на 1 работа антивирус далёкий н...,1
4,5,"Очень удобно, работает быстро.",2017-08-14,очень удобно работать быстро,1


In [9]:
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['target'], test_size=0.2,
                                                    random_state=13, stratify=data['target'])

In [10]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [11]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /home/ai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
tokens = word_tokenize(train_corpus)

In [13]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [14]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [15]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [16]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [17]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [18]:
x_train

array([[  0,   0,   0, ...,   0,   0, 267],
       [  0,   0,   0, ...,   8, 998,  20],
       [  0,   0,   0, ...,  19,  53, 480],
       ...,
       [  0,   0,   0, ...,   0,   0,   5],
       [  0,   0,   0, ...,   0,   0,  22],
       [  0,   0,   0, ...,   0,   2,  19]], dtype=int32)

# Keras model

In [19]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [20]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_test, num_classes)

In [21]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
28/28 [==============================] - 3s 121ms/step - loss: 0.4113 - accuracy: 0.8473 - val_loss: 0.3069 - val_accuracy: 0.8430
Epoch 2/10
28/28 [==============================] - 0s 15ms/step - loss: 0.2414 - accuracy: 0.8473 - val_loss: 0.2000 - val_accuracy: 0.8430
Epoch 3/10
28/28 [==============================] - 0s 14ms/step - loss: 0.1886 - accuracy: 0.8818 - val_loss: 0.1874 - val_accuracy: 0.9196
Epoch 4/10
28/28 [==============================] - 0s 15ms/step - loss: 0.1716 - accuracy: 0.9394 - val_loss: 0.1867 - val_accuracy: 0.9228
Epoch 5/10
28/28 [==============================] - 0s 15ms/step - loss: 0.1604 - accuracy: 0.9467 - val_loss: 0.1932 - val_accuracy: 0.9203


In [24]:
score = model.evaluate(x_test, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

8/8 [==============================] - 1s 101ms/step - loss: 0.1908 - accuracy: 0.9251


Test score: 0.19084373116493225
Test accuracy: 0.9250633120536804


In [25]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

8/8 [==============================] - 0s 4ms/step


In [26]:
roc_auc_score(y_test, results[:, 1])

0.9625255407725852

### RNN

In [27]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [28]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train[:, 1],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
28/28 [==============================] - 2s 58ms/step - loss: 0.4588 - accuracy: 0.7993 - val_loss: 0.3082 - val_accuracy: 0.8646
Epoch 2/10
28/28 [==============================] - 1s 51ms/step - loss: 0.2827 - accuracy: 0.8906 - val_loss: 0.2318 - val_accuracy: 0.9025
Epoch 3/10
28/28 [==============================] - 1s 52ms/step - loss: 0.2047 - accuracy: 0.9235 - val_loss: 0.1892 - val_accuracy: 0.9127
Epoch 4/10
28/28 [==============================] - 1s 52ms/step - loss: 0.1681 - accuracy: 0.9355 - val_loss: 0.1946 - val_accuracy: 0.9139


In [29]:
score = model.evaluate(x_test, y_val[:, 1], batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results = model.predict(x_test, batch_size=batch_size, verbose=1)

roc_auc_score(y_test, results)

8/8 [==============================] - 0s 10ms/step - loss: 0.1915 - accuracy: 0.9182


Test score: 0.1915275901556015
Test accuracy: 0.9182278513908386
8/8 [==============================] - 0s 11ms/step


0.9577034923223263

### LSTM

In [30]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train[:, 1],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
28/28 [==============================] - 2s 59ms/step - loss: 0.4509 - accuracy: 0.8380 - val_loss: 0.2972 - val_accuracy: 0.8551
Epoch 2/10
28/28 [==============================] - 1s 26ms/step - loss: 0.2732 - accuracy: 0.8889 - val_loss: 0.2240 - val_accuracy: 0.8924
Epoch 3/10
28/28 [==============================] - 1s 25ms/step - loss: 0.2068 - accuracy: 0.9127 - val_loss: 0.1876 - val_accuracy: 0.9139
Epoch 4/10
28/28 [==============================] - 1s 25ms/step - loss: 0.1619 - accuracy: 0.9354 - val_loss: 0.1816 - val_accuracy: 0.9234
Epoch 5/10
28/28 [==============================] - 1s 25ms/step - loss: 0.1429 - accuracy: 0.9453 - val_loss: 0.1815 - val_accuracy: 0.9241
Epoch 6/10
28/28 [==============================] - 1s 26ms/step - loss: 0.1293 - accuracy: 0.9520 - val_loss: 0.1836 - val_accuracy: 0.9272


In [32]:
score = model.evaluate(x_test, y_val[:, 1], batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results = model.predict(x_test, batch_size=batch_size, verbose=1)

roc_auc_score(y_test, results)

8/8 [==============================] - 0s 8ms/step - loss: 0.1806 - accuracy: 0.9271


Test score: 0.1805633157491684
Test accuracy: 0.9270886182785034
8/8 [==============================] - 0s 8ms/step


0.9603570149106228

### GRU

In [33]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(GRU(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train[:, 1],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
28/28 [==============================] - 2s 63ms/step - loss: 0.4941 - accuracy: 0.8349 - val_loss: 0.3009 - val_accuracy: 0.8430
Epoch 2/10
28/28 [==============================] - 1s 25ms/step - loss: 0.2667 - accuracy: 0.8761 - val_loss: 0.2204 - val_accuracy: 0.8981
Epoch 3/10
28/28 [==============================] - 1s 25ms/step - loss: 0.2046 - accuracy: 0.9154 - val_loss: 0.1980 - val_accuracy: 0.9025
Epoch 4/10
28/28 [==============================] - 1s 25ms/step - loss: 0.1665 - accuracy: 0.9327 - val_loss: 0.1868 - val_accuracy: 0.9234
Epoch 5/10
28/28 [==============================] - 1s 25ms/step - loss: 0.1465 - accuracy: 0.9448 - val_loss: 0.2010 - val_accuracy: 0.9253


In [35]:
score = model.evaluate(x_test, y_val[:, 1], batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

results = model.predict(x_test, batch_size=batch_size, verbose=1)

roc_auc_score(y_test, results)

8/8 [==============================] - 0s 8ms/step - loss: 0.2034 - accuracy: 0.9170


Test score: 0.20343713462352753
Test accuracy: 0.9169620275497437
8/8 [==============================] - 0s 8ms/step


0.9578840998653473

### LSTM немного опережает RNN и GRU.